In [1]:
import requests
from bs4 import BeautifulSoup as BS
import pandas as pd
import re
import numpy as np

In [2]:
candidate_df = pd.DataFrame()

In [7]:
def district_scraper(state, district):
    URL = f'https://www.opensecrets.org/races/candidates?cycle=2020&id={state}{district}&spec=N'
    soup = BS(requests.get(URL).text)
    
    candidate_list = [x.text.strip() for x in soup.find_all('strong')]
    finances = (pd
        .read_html(
            str(soup.findAll('table', attrs={'class' : 'Members--table'}))
                .replace('$','')
                .replace(':', '')
        )
    )
    
    
    temp_df = pd.DataFrame()
    
    temp_df['name'] = [re.findall(r'(.+ .+) \(\w', item)[0] for item in candidate_list]
    temp_df['party'] = [re.findall(r'\((\w)\)', item)[0] for item in candidate_list]
    temp_df['vote'] = [re.findall(r'(\d*[.?]\d*\%)', item)[0] for item in candidate_list]
    temp_df['winner'] = ['Winner' if re.search(r'(Winner)', str(item)) is not None
                              else 'Not Winner' for item in candidate_list]
    temp_df['incumbent'] = ['Incumbent' if re.search(r'(Incumbent)', str(item)) != None 
                            else 'Not Incumbent' for item in candidate_list]
    temp_df['state'] = str(re.search(r'[A-Z][a-z]+(?: [A-Z][a-hj-z][a-z]+)?',
                                     soup.findAll('title')[0].text)[0])
    temp_df['district'] = str(re.findall(r'District ([0-9]{2})', soup.findAll('title')[0].text)[0])
    temp_df['raised'] = [finances[i].iloc[0,1] for i in range(len(candidate_list))]
    temp_df['spent'] = [finances[i].iloc[1,1] for i in range(len(candidate_list))]
    temp_df['cash_on_hand'] = [finances[i].iloc[2,1] for i in range(len(candidate_list))]
    
    global candidate_df
    candidate_df = pd.concat([candidate_df, temp_df])
    candidate_df = candidate_df.reset_index(drop=True)

In [8]:
candidate_df = pd.DataFrame()

for num in range(1,10):
    district_scraper('TN', str(num).zfill(2))

In [9]:
candidate_df

,name,party,vote,winner,incumbent,state,district,raised,spent,cash_on_hand
0,Diana Harshbarger,R,74.8%,Winner,Not Incumbent,Tennessee,01,2126946,1869100,257846
1,Blair Nicole Walsingham,D,22.4%,Not Winner,Not Incumbent,Tennessee,01,140209,134995,5215
2,Tim Burchett,R,67.7%,Winner,Incumbent,Tennessee,02,1336276,878488,593678
3,Renee Hoyos,D,31.0%,Not Winner,Not Incumbent,Tennessee,02,812784,816793,210
4,Chuck Fleischmann,R,67.3%,Winner,Incumbent,Tennessee,03,1051653,381411,1880341
5,Meg Gorman,D,30.5%,Not Winner,Not Incumbent,Tennessee,03,85843,77760,8083
6,Scott Desjarlais,R,66.7%,Winner,Incumbent,Tennessee,04,331464,392499,302649
7,Christopher Hale,D,33.3%,Not Winner,Not Incumbent,Tennessee,04,308731,302996,5735
8,Jim Cooper,D,100.0%,Winner,Incumbent,Tennessee,05,936569,1332131,272934
9,John Rose,R,73.7%,Winner,Incumbent,Tennessee,06,1050429,625688,454375


In [10]:
candidate_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 19 entries, 0 to 18
Data columns (total 10 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   name          19 non-null     object
 1   party         19 non-null     object
 2   vote          19 non-null     object
 3   winner        19 non-null     object
 4   incumbent     19 non-null     object
 5   state         19 non-null     object
 6   district      19 non-null     object
 7   raised        19 non-null     int64 
 8   spent         19 non-null     int64 
 9   cash_on_hand  19 non-null     int64 
dtypes: int64(3), object(7)
memory usage: 1.6+ KB


In [11]:
district_url = 'https://www.britannica.com/topic/United-States-House-of-Representatives-Seats-by-State-1787120'
district_soup = BS(requests.get(district_url).text)
district_df = pd.read_html(str(district_soup.find('table')))[0]
district_df

,state,representatives
0,Alabama,7
1,Alaska,1
2,Arizona,9
3,Arkansas,4
4,California,53
5,Colorado,7
6,Connecticut,5
7,Delaware,1
8,Florida,27
9,Georgia,14


In [12]:
state_url = 'https://www.50states.com/abbreviations.htm'
state_soup = BS(requests.get(state_url).text)

state_df = (
    pd.read_html(str(state_soup.find('table', 
                                            attrs={'class' : 'table table-hover'})))[0]
    .drop(columns = 'STANDARD ABBREVIATION')
    .rename(columns = {'US STATE' : 'state',
                       'POSTAL ABBREVIATION' : 'abbrev'})
)
state_df

,state,abbrev
0,Alabama,AL
1,Alaska,AK
2,Arizona,AZ
3,Arkansas,AR
4,California,CA
5,Colorado,CO
6,Connecticut,CT
7,Delaware,DE
8,Florida,FL
9,Georgia,GA


In [13]:
state_details = pd.merge(state_df, district_df, how='inner', on='state')
state_details

,state,abbrev,representatives
0,Alabama,AL,7
1,Alaska,AK,1
2,Arizona,AZ,9
3,Arkansas,AR,4
4,California,CA,53
5,Colorado,CO,7
6,Connecticut,CT,5
7,Delaware,DE,1
8,Florida,FL,27
9,Georgia,GA,14


In [14]:
district_dictionary = state_details.set_index('abbrev')['representatives'].to_dict()
district_dictionary

{'AL': 7,
 'AK': 1,
 'AZ': 9,
 'AR': 4,
 'CA': 53,
 'CO': 7,
 'CT': 5,
 'DE': 1,
 'FL': 27,
 'GA': 14,
 'HI': 2,
 'ID': 2,
 'IL': 18,
 'IN': 9,
 'IA': 4,
 'KS': 4,
 'KY': 6,
 'LA': 6,
 'ME': 2,
 'MD': 8,
 'MA': 9,
 'MI': 14,
 'MN': 8,
 'MS': 4,
 'MO': 8,
 'MT': 1,
 'NE': 3,
 'NV': 4,
 'NH': 2,
 'NJ': 12,
 'NM': 3,
 'NY': 27,
 'NC': 13,
 'ND': 1,
 'OH': 16,
 'OK': 5,
 'OR': 5,
 'PA': 18,
 'RI': 2,
 'SC': 7,
 'SD': 1,
 'TN': 9,
 'TX': 36,
 'UT': 4,
 'VT': 1,
 'VA': 11,
 'WA': 10,
 'WV': 3,
 'WI': 8,
 'WY': 1}

In [ ]:
# another way to iterate through the financial data, pulling pivoted values and concatting them
# intitialize empty data frame
final_finances = pd.DataFrame()

# create the list of all dataframe tables for candidates
finances = (pd
    .read_html(
        str(soup_TN07.findAll('table', attrs={'class' : 'Members--table'}))
            .replace('$','')
            .replace(':', '')
    )
)

for fin in finances:
    final_finances = pd.concat(
        [final_finances,
        fin.pivot(index=2, columns=0, values=1).reset_index(drop=True)])